In [ ]:

# In[10]:


# def plot_concentrations(y):
#     fig, axs = plt.subplots(2, 2, sharey=False, tight_layout=True)
#     for ip in range(0,4):
#         axs[np.unravel_index(ip, axs.shape)].hist(y[:,ip])
# plot_concentrations(y_test)
# plot_concentrations(y_pred)

def plot_compare_test_pred(yt, yp):
    fig, axs = plt.subplots(2, 4, sharey=False, tight_layout=True, figsize=(12, 5))
    for ip in range(0,8):
        axsidx = np.unravel_index(ip, axs.shape)
        if ip < 4:
            axs[axsidx].hist(yt[:,ip])
        else:
            axs[axsidx].hist(yp[:,ip-4])

plot_compare_test_pred(yt, yp)
plt.show()


# In[11]:


# best_model.summary()


# In[12]:


model = best_model

idxs = np.random.permutation(x_val.shape[0])
x_cam = x_val[idxs[:1000],:,:]
y_cam = y_val[idxs[:1000], :]

w_k_c = model.layers[-1].get_weights()[0] # weights for each filter k for each class c 

new_input_layer = model.inputs # same input of the original model

new_outpu_layer = [model.get_layer("conv1d_31").output, model.layers[-1].output] # output is both the original as well as the before last layer 
# new_outpu_layer = [model.get_layer("activation_8").output, model.layers[-1].output] # output is both the original as well as the before last layer 

new_function = keras.backend.function(new_input_layer,new_outpu_layer)

new_feed_forward = new_function

[conv_out, y_pred] = new_feed_forward((x_cam,))

metrics = classifier._calculate_metrics(y_cam, y_pred, 0.0)
display(HTML(metrics.to_html()))
print(conv_out.shape)


# In[13]:


print(w_k_c.shape)
print(conv_out.shape)
print(x_cam.shape)


# In[14]:


# plot_data = []
# for isample in range(conv_out.shape[0]):
#     conv_out_i = conv_out[isample,:,:]
#     one_plot_data = []
#     for j in range(y_val.shape[1]):

#         axsidx = np.unravel_index(j, axs.shape)

#         cas = np.zeros(dtype=np.float, shape = (conv_out.shape[1]))
#         for k,w in enumerate(w_k_c[:,j]):
#             cas += w * conv_out_i[:,k]
#         minimum = np.min(cas)
#         cas = cas - minimum
#         cas = cas/max(cas)
#         cas = cas * 100
#         cas = cas.astype(int)

#     #     axs[axsidx].plot(cas)

#         one_plot_data.append( (np.squeeze(x_cam[isample,:,:]), np.array(range(y.shape[0])), cas) )
#     plot_data.append(one_plot_data)

# import ipywidgets as widgets
# from matplotlib.collections import LineCollection
# from matplotlib.colors import ListedColormap, BoundaryNorm

# plt.ioff()
# fig, axs = plt.subplots(2, 2, sharey=False, tight_layout=True, figsize=(12, 5))
# plt.ion()

# def show_image(one_plot_data):
#     for j in range(y_val.shape[1]):

#         axsidx = np.unravel_index(j, axs.shape)
# #         axs[axsidx].clear()

#         y = one_plot_data[j][0]
#         x = one_plot_data[j][1]
#         dydx = one_plot_data[j][2]

#         # def plot_line_multicolor(x, y, dydx):
#         # Create a set of line segments so that we can color them individually
#         # This creates the points as a N x 1 x 2 array so that we can stack points
#         # together easily to get the segments. The segments array for line collection
#         # needs to be (numlines) x (points per line) x 2 (for x and y)
#         points = np.array([x, y]).T.reshape(-1, 1, 2)
#         segments = np.concatenate([points[:-1], points[1:]], axis=1)

#         # Create a continuous norm to map from data points to colors
#         norm = plt.Normalize(dydx.min(), dydx.max())
#         lc = LineCollection(segments, cmap='jet', norm=norm)
#         # Set the values used for colormapping
#         lc.set_array(dydx)
#         lc.set_linewidth(4)
#         line = axs[axsidx].add_collection(lc)
# #         fig.colorbar(line, ax=axs[axsidx])
        
#         axs[axsidx].set_xlim(x.min(), x.max())
#         axs[axsidx].set_ylim(y.min(), y.max())
#         axs[axsidx].set_title(names[j])

# show_image(plot_data[0])



# def update(change):
#     idx = change['new']
#     show_image(plot_data[idx])
#     fig.canvas.draw_idle()
    
# slider = widgets.IntSlider(value=0, min=0, max=conv_out.shape[0]-1)
# slider.observe(update, names='value')
# widgets.VBox([slider, fig.canvas])

# # plt.show()


# In[15]:


from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

# print("original_label: "+str(encoder.inverse_transform(np.argmax(original_binary_class))))
# print("original_shape: "+str(time_series_original.shape))
# print("predicted_label:"+str(encoder.inverse_transform(np.argmax(predicted))))
# print("predicted_shape:"+str(conv_out.shape))

# print(w_k_c.shape)
# print(conv_out.shape)

fig, axs = plt.subplots(2, 2, sharey=False, tight_layout=True, figsize=(12, 5))

ix = np.random.randint(0,conv_out.shape[0]-1)
conv_out_0 = conv_out[ix,:,:]
# conv_out_avg = np.squeeze(np.mean(conv_out, axis=0))

for j in range(y_val.shape[1]):

    axsidx = np.unravel_index(j, axs.shape)

    cas = np.zeros(dtype=np.float, shape = (conv_out.shape[1]))
    for k,w in enumerate(w_k_c[:,j]):
        cas += w * conv_out_0[:,k]
    minimum = np.min(cas)
    cas = cas - minimum
    cas = cas/max(cas)
    cas = cas * 100
    cas = cas.astype(int)

#     axs[axsidx].plot(cas)

    y = np.squeeze(x_cam[ix,:,:])
    x = np.array(range(y.shape[0]))
    dydx = cas

    # Create a set of line segments so that we can color them individually
    # This creates the points as a N x 1 x 2 array so that we can stack points
    # together easily to get the segments. The segments array for line collection
    # needs to be (numlines) x (points per line) x 2 (for x and y)
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    # Create a continuous norm to map from data points to colors
    norm = plt.Normalize(dydx.min(), dydx.max())
    lc = LineCollection(segments, cmap='jet', norm=norm)
    # Set the values used for colormapping
    lc.set_array(dydx)
    lc.set_linewidth(4)
    line = axs[axsidx].add_collection(lc)
    fig.colorbar(line, ax=axs[axsidx])

    # # Use a boundary norm instead
    # cmap = ListedColormap(['r', 'g', 'b'])
    # norm = BoundaryNorm([-1, -0.5, 0.5, 1], cmap.N)
    # lc = LineCollection(segments, cmap=cmap, norm=norm)
    # lc.set_array(dydx)
    # lc.set_linewidth(2)
    # line = axs[1].add_collection(lc)
    # fig.colorbar(line, ax=axs[1])

    axs[axsidx].set_xlim(x.min(), x.max())
    axs[axsidx].set_ylim(y.min(), y.max())
    axs[axsidx].set_title(names[j])

plt.show()




In [1]:
import os

In [4]:

jitter = None
# jitter = 500
epochs = 100
hold_probe = 'CF025'
output_directory = f'/mnt/nfs/proj/in-vitro/Leonardo/inception/results/cf/J{jitter}/E{epochs}/{hold_probe}/'

if not (os.path.exists(output_directory)):
    os.makedirs(output_directory, exist_ok=True)